In [1]:
import gc
gc.collect()  # Force garbage collection

3

In [3]:
!pip install psutil


In [13]:
import psutil

# Get memory information
virtual_memory = psutil.virtual_memory()

# Display available memory
print(f"Total Memory: {virtual_memory.total / (1024**3):.2f} GB")
print(f"Available Memory: {virtual_memory.available / (1024**3):.2f} GB")
print(f"Used Memory: {virtual_memory.used / (1024**3):.2f} GB")
print(f"Memory Percentage: {virtual_memory.percent}%")


Total Memory: 7.66 GB
Available Memory: 2.71 GB
Used Memory: 4.95 GB
Memory Percentage: 64.6%


In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.utils import to_categorical

In [5]:
# Step 3: Dataset Preparation
# Function to load dataset directly into memory (optional, for smaller datasets)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def load_dataset(path, img_size=(224, 224)):
    images = []
    labels = []
    class_names = sorted(os.listdir(path))  # Ensure consistent label ordering

    for class_name in class_names:
        class_path = os.path.join(path, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)

            # Skip hidden/system files
            if img_name.startswith('.'):
                continue

            # Read and preprocess the image
            img = cv2.imread(img_path)
            if img is None:
                print(f"Warning: Unable to read image {img_path}. Skipping.")
                continue

            img = cv2.resize(img, img_size).astype('float32') / 255.0  # Normalize
            images.append(img)
            labels.append(class_names.index(class_name))  # Assign numeric label

    return np.array(images, dtype='float32'), np.array(labels, dtype='int32')

# For large datasets, use a generator to avoid memory issues
def get_data_generator(path, img_size=(224, 224), batch_size=32):
    datagen = ImageDataGenerator(rescale=1.0 / 255.0)  # Normalize
    generator = datagen.flow_from_directory(
        path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True  # Shuffle for better training
    )
    return generator

# Paths to your datasets
train_path = r"D:\PFE\Datasets\ForClassification\wdd2017\Training"
test_path = r"D:\PFE\Datasets\ForClassification\wdd2017\Testing"
val_path = r"D:\PFE\Datasets\ForClassification\wdd2017\Validation"


# **Option 2: Use data generators for large datasets**
img_size = (224, 224)
batch_size = 32  # Adjust based on available memory
train_generator = get_data_generator(train_path, img_size, batch_size)
test_generator = get_data_generator(test_path, img_size, batch_size)
val_generator = get_data_generator(val_path, img_size, batch_size)

Found 3841 images belonging to 5 classes.
Found 693 images belonging to 5 classes.
Found 987 images belonging to 5 classes.


In [ ]:
#print

In [14]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

def build_fcn(input_shape=(224, 224, 3), num_classes=5):
    """
    Build the Fully Convolutional Network (FCN) model for classification.
    """
    inputs = Input(shape=input_shape)
   
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
   
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
   
    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
   
    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
   
    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
   
    # Fully Convolutional Layers
    x = Conv2D(1024, (6, 6), activation='relu', padding='valid', name='fc6')(x)
    x = Conv2D(1024, (1, 1), activation='relu', padding='valid', name='fc7')(x)
   
    # Global Average Pooling to reduce spatial dimensions to 1x1
    x = GlobalAveragePooling2D()(x)
   
    # Final Dense layer for classification
    x = Dense(num_classes, activation='softmax', name='predictions')(x)
   
    model = Model(inputs, x, name='FCN')
    return model

# Build the model
model = build_fcn(input_shape=(224, 224, 3), num_classes=5)
model.summary()

Model: "FCN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc6 (Conv2D)                         │ (None, 2, 2, 1024)          │      18,875,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc7 (Conv2D)                         │ (None, 2, 2, 1024)          │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 34,644,805 (132.16 MB)

 Trainable params: 34,644,805 (132.16 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
optimizer = SGD(learning_rate=0.00005, momentum=0.9, nesterov=True)
loss_fn = CategoricalCrossentropy(from_logits=False)  # Match with your output
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,  # Ensure the generator outputs (x, y) with matching shapes
    validation_data=val_generator,
    epochs=30,
    verbose=1
)

Epoch 1/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 2627s 22s/step - accuracy: 0.2469 - loss: 1.6085 - val_accuracy: 0.3161 - val_loss: 1.6041
Epoch 2/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 2611s 22s/step - accuracy: 0.2906 - loss: 1.6046 - val_accuracy: 0.3161 - val_loss: 1.5994
Epoch 3/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 2602s 22s/step - accuracy: 0.2694 - loss: 1.6018 - val_accuracy: 0.3161 - val_loss: 1.5948
Epoch 4/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 2604s 22s/step - accuracy: 0.2876 - loss: 1.5977 - val_accuracy: 0.3161 - val_loss: 1.5904
Epoch 5/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 2927s 24s/step - accuracy: 0.2851 - loss: 1.5957 - val_accuracy: 0.3161 - val_loss: 1.5863
Epoch 6/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 3239s 27s/step - accuracy: 0.2839 - loss: 1.5928 - val_accuracy: 0.3161 - val_loss: 1.5822
Epoch 7/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 3210s 27s/step - accuracy: 0.2849 - loss: 1.5893 - val_accuracy: 0.3161 - val_loss: 1.5782
Epoch 8/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 3160s 26s/step - accuracy: 0.2907 - loss: 1

In [ ]:
# Step 4: Evaluation
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_acc:.4f}")


In [ ]:
# Step 5: Localization Using Bounding Box Approximation (BBA)
def bounding_box_approximation(score_map, threshold=0.5):
    """
    Perform Bounding Box Approximation (BBA) to localize disease areas.
    """
    # Threshold the score map
    _, binary_map = cv2.threshold(score_map, threshold, 1, cv2.THRESH_BINARY)
   
    # Find contours
    contours, _ = cv2.findContours(binary_map.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
   
    # Generate bounding boxes
    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append((x, y, x + w, y + h))
   
    return bounding_boxes

# Example usage for localization
score_map = model.predict(X_test[0:1])[0]  # Get score map for the first test image
bounding_boxes = bounding_box_approximation(score_map[..., 0])  # Use the first channel (class)
print("Bounding Boxes:", bounding_boxes)

# Visualize the bounding boxes on the image
img = (X_test[0] * 255).astype(np.uint8)
for (x1, y1, x2, y2) in bounding_boxes:
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

cv2.imshow("Localization", img)
cv2.waitKey(0)
cv2.destroyAllWindows()